In [2]:
import dotenv
dotenv.load_dotenv(override=True)

import os
from typing import List, Union
from agno.tools import tool
#from agno.team import Team
from agno.workflow import Workflow
from agno.agent import Agent
from agno.tools.mcp import MCPTools
from agno.storage.json import JsonStorage
#from agno.models.litellm import LiteLLM
from agno.models.openai.like import OpenAILike

import asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Initialize LiteLLM/ OpenAI-like model
# model = LiteLLM(
#     id = "openai/gemma3:27b-it-qat", # provider need to be added before model name
#     api_base="https://ml.gss.com.tw/gemma3",
#     api_key="sk-test-key",
# )

model = OpenAILike(
    id = "openai/gemma3:27b-it-qat", # provider need to be added before model name
    base_url=os.getenv("AGENT_BASE_URL"),
    api_key=os.getenv("AGENT_API_KEY")
)

In [ ]:
# MCP server
rag_mcp_command = "python3 /agent_framework_demo/demo/1_rag_server.py stdio" #TODO: 要使用絕對路徑

rag_mcp_server = MCPTools(
    command=rag_mcp_command,
    env={
        "OPENAI_BASE_URL": os.getenv("OPENAI_BASE_URL"),
        "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
    },
    timeout_seconds=90,
)

In [5]:
# structured output
from pydantic import BaseModel, Field

class Reference(BaseModel):
    score: float
    #relevant: bool
    text: str

class References(BaseModel):
    references: List[Reference] = Field(
        default_factory=list,
        description="List of passages retrieved from database."
    )

class ReferenceWithCriticism(BaseModel):
    relevant: bool
    text: str
    
class Criticism(BaseModel):
    passages: List[ReferenceWithCriticism]
    explanation: str
    sufficient: bool

class RAGFormat(BaseModel):
    reference: List[str] = Field(
        default_factory=list,
        description="List of passages text used to generate the answer."
    )
    final_answer: str


In [6]:
# Agent initialization
storage_path = f"agno_storage"

assistant_agent = Agent(
    name="assistant_agent",
    model = model,
    instructions = (
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Direct Answering: If the question is clear and within your knowledge, answer directly.\n"
        "2. Clarification: If the question is vague or unclear, ask clarifying questions to understand the user’s intent before responding.\n"
        "3. Retrieval: If the question requires specialized or external knowledge, use the `rag` to obtain an answer based on relevant information retrieved from the database. Return the result with `RAGFormat` format.\n"
    ),
    storage=JsonStorage(dir_path=storage_path),
    add_history_to_messages=True,
    num_history_runs=3,

)
seeker_agent = Agent(
    name="seeker_agent",
    model=model,
    instructions=(
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Retrieve relevant documents from the specific vector database. When retrieving from the database, the user's original intent should be preserved as much as possible, and the clarity of the question's meaning should be maintained.\n"
        "2. Return all the retrieval results with Reference format.\n"
    ),
    storage=JsonStorage(dir_path=storage_path),
    add_history_to_messages=True,
    num_history_runs=3,
    #response_model=References,
    #parse_response = True
)

critic_agent = Agent(
    name="critic_agent",
    model=model, 
    instructions=(
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Critically evaluate whether the retrieved passages are relevant to the user's query.\n"
        "2. Critically evaluate whether the retrieved passages are sufficient to answer the user's query.\n"
    ),
    storage=JsonStorage(dir_path=storage_path),
    add_history_to_messages=True,
    num_history_runs=3,
    response_model=Criticism,
    parse_response = True
)
generator_agent = Agent(
    name="generator_agent",
    model=model,
    instructions=(
        "You are a helpful and intelligent assistant. The user you are helping speaks Traditional Chinese and comes from Taiwan, so in most cases, you should respond in Traditional Chinese. \n"
        "Behavior Rules: \n"
        "1. Generate a final answer based on the retrieved passages.\n"
        "2. If the retrieved passages are insufficient, ask follow-up questions to gather more information.\n"
    ),
    storage=JsonStorage(dir_path=storage_path),
    add_history_to_messages=True,
    num_history_runs=3,
    response_model=RAGFormat,
    respond_directly = True,
    parse_response = True
)

In [7]:
# multi-turn conversation
async def multi_turn_conversation(start_agent: Union[Workflow, Agent], user_id, session_id):
    while True:
        user_input = input("You: ")
        print(f"You: {user_input}")

        if user_input == "exit":
            print("Agent: Goodbye!")
            return result
        
        result = await start_agent.arun(
            message = user_input,
            user_id = user_id,
            session_id = session_id,
        )
        print(f"Agent: {result.content}")
    

# Workflow

In [9]:
# Workflow
class RAG(Workflow):
    name = "RAG"
    seeker: Agent = seeker_agent
    critic: Agent = critic_agent
    generator: Agent = generator_agent
    mcp_tools = rag_mcp_server

    async def arun(self, message: str, user_id: str, session_id) -> Union[RAGFormat, str]:
        retrieved_passages = await self.retrieve_info(message, user_id, session_id)
        critic_result = await self.get_passage_criticism(message, retrieved_passages, user_id, session_id)
        # 因為workflow最後不會再過一次LLM，所以先直接輸出生成的結果，不根據critic_result類別決定是否放入generate_answer()
        #if critic_result.sufficient:
        final_answer = await self.generate_answer(message, critic_result, user_id, session_id)
        return final_answer
        # else:
        #     print("Insufficient criticism from the critic agent.")
        #     return critic_result
    
    async def retrieve_info(self, query: str, user_id: str, session_id: str):
        n_retries = 3
        # async with self.mcp_tools as mcp_tools:
        #     self.seeker.tools = [mcp_tools]
        for _ in range(n_retries):
            print("start seeker agent...")
            response = await self.seeker.arun(
                message = query,
                user_id = user_id,
                session_id = session_id,
            )
            print(f"seeker agent: {response.content}")
            return response.content
            #TODO: 如果有順利在seeker加上structured output，可以試試看這段
            # try:
            #     if isinstance(response.content, References):
            #         print("seeker agent:", response.content)
            #         return response.content
            # except Exception as e:
            #     print(f"Error in seeker agent response: {e}")

    async def get_passage_criticism(self, user_query: str, retrieved_passages: List[Reference], user_id: str, session_id: str):
        """
            Get criticism for each retrieved passage.
        """
        input_text = (
            "The user's query and the retrieved passages are as follows:\n"
            f"## User Query: {user_query}\n"
            f"## Retrieved Passages:\n{retrieved_passages}"
            #TODO: 如果有順利在seeker加上structured output，可以試試看這段，然後移除上面的{retrieved_passages}
            #+ "\n".join([f"[{i + 1}] {passage.text}" for i, passage in enumerate(retrieved_passages)])

        )
        n_retries = 3
        for _ in range(n_retries):
            response = await self.critic.arun(
                message = input_text,
                user_id = user_id,
                session_id = session_id,
            )
            if isinstance(response.content, Criticism):
                print("critic agent:", response.content)
                return response.content


    async def generate_answer(self, user_query: str, passages_with_criticism: Criticism, user_id: str, session_id: str):
        """
            Generate final answer to the user query based on retrieved passages.
        """
        input_text = (
            "The user's query and the retrieved passages are as follows:\n"
            f"## User Query: {user_query}\n"
            "## Relevant Passages:\n"
            + "\n".join([f"[{i+1}] {passage.text}" for i, passage in enumerate(passages_with_criticism.passages) if passage.relevant])
            #TODO: 如果有順利在seeker加上structured output，可以試試看這段，然後移除上面的{retrieved_passages}
            #+ "\n".join([f"[{i + 1}] {passage.text}" for i, passage in enumerate(retrieved_passages)])
        )
        n_retries = 3
        for _ in range(n_retries):
            response = await self.generator.arun(
                message=input_text,
                user_id=user_id,
                session_id=session_id,
            )
            if isinstance(response.content, RAGFormat):
                print("generator agent:", response.content)
                return response # workflow 只能 return RunResponse objects

In [10]:
async with rag_mcp_server as mcp_tools:
    rag_workflow = RAG()
    rag_workflow.seeker.tools = [mcp_tools]
    result = asyncio.run(multi_turn_conversation(rag_workflow, "user_1", "session_6"))

You: 我想要請假，有沒有什麼限制？
start seeker agent...
seeker agent: 您好，關於請假限制，根據公司規定（參考以下資訊）：

*   **工作時間：** 每週工作時數不得超過四十小時，每日工作時數為八小時。
*   **請假種類：** 公司有特休假、生日假、事假、病假等多種休假種類。
*   **請假流程：** 請假需事先填寫請假單，並依請假天數不同，經不同層級主管核定。
    *   5天以內：需提前3天向部門主管請假。
    *   5-10天：需提前1週向處級主管請假。
    *   10天以上：需提前2週向業務/事業群總經理請假。
*   **臨時請假：** 突發狀況可於當日10:00前電話報備，事後補辦手續。
*   **未依規定請假：** 未依規定請假且未補辦手續，視同曠職。

詳細規定請參考公司內部規定，或向人資部門詢問。希望這些資訊對您有幫助！
critic agent: passages=[ReferenceWithCriticism(relevant=True, text='您好，關於請假限制，根據公司規定（參考以下資訊）：\n\n*   **工作時間：** 每週工作時數不得超過四十小時，每日工作時數為八小時。\n*   **請假種類：** 公司有特休假、生日假、事假、病假等多種休假種類。\n*   **請假流程：** 請假需事先填寫請假單，並依請假天數不同，經不同層級主管核定。\n    *   5天以內：需提前3天向部門主管請假。\n    *   5-10天：需提前1週向處級主管請假。\n    *   10天以上：需提前2週向業務/事業群總經理請假。\n*   **臨時請假：** 突發狀況可於當日10:00前電話報備，事後補辦手續。\n*   **未依規定請假：** 未依規定請假且未補辦手續，視同曠職。\n\n詳細規定請參考公司內部規定，或向人資部門詢問。希望這些資訊對您有幫助！')] explanation="The retrieved passage directly addresses the user's question about the limitations of taking leave. It provides a clear summary of the company'

# Work Around: function

In [8]:
async def retrieve_info(query: str):
    n_retries = 3
    #async with rag_mcp_server as mcp_tools:
        #seeker_agent.tools = [mcp_tools]
    for _ in range(n_retries):
        print("start seeker agent...")
        response = await seeker_agent.arun(
            message = query,
        )
        print(f"seeker agent: {response.content}")
        return response.content
        #TODO: 如果有順利在seeker加上structured output，可以試試看這段
        # try:
        #     if isinstance(response.content, References):
        #         print("seeker agent:", response.content)
        #         return response.content
        # except Exception as e:
        #     print(f"Error in seeker agent response: {e}")

async def get_passage_criticism(user_query: str, retrieved_passages: List[Reference]):
    """
        Get criticism for each retrieved passage.
    """
    input_text = (
        "The user's query and the retrieved passages are as follows:\n"
        f"## User Query: {user_query}\n"
        f"## Retrieved Passages:\n{retrieved_passages}"
        #TODO: 如果有順利在seeker加上structured output，可以試試看這段，然後移除上面的{retrieved_passages}
        #+ "\n".join([f"[{i + 1}] {passage.text}" for i, passage in enumerate(retrieved_passages)])

    )
    n_retries = 3
    for _ in range(n_retries):
        response = await critic_agent.arun(
            message = input_text,
        )
        if isinstance(response.content, Criticism):
            print("critic agent:", response.content)
            return response.content


async def generate_answer(user_query: str, passages_with_criticism: Criticism):
    """
        Generate final answer to the user query based on retrieved passages.
    """
    input_text = (
        "The user's query and the retrieved passages are as follows:\n"
        f"## User Query: {user_query}\n"
        "## Relevant Passages:\n"
        + "\n".join([f"[{i+1}] {passage.text}" for i, passage in enumerate(passages_with_criticism.passages) if passage.relevant])
        #TODO: 如果有順利在seeker加上structured output，可以試試看這段，然後移除上面的{retrieved_passages}
        #+ "\n".join([f"[{i + 1}] {passage.text}" for i, passage in enumerate(retrieved_passages)])
    )
    n_retries = 3
    for _ in range(n_retries):
        response = await generator_agent.arun(
            message=input_text,
        )
        if isinstance(response.content, RAGFormat):
            print("generator agent:", response.content)
            return response.content
    
@tool(
    description="response the user query based on external knowledge",  # Custom description (otherwise the function docstring is used)
    show_result=True,                               # Show result after function call
    #stop_after_tool_call=True, 
)
async def rag(query: str):
    """
        response the user query based on external knowledge
    """
    retrieved_passages = await retrieve_info(query)
    critic_result = await get_passage_criticism(query, retrieved_passages)
    if critic_result.sufficient:
        final_answer = await generate_answer(query, critic_result)
        return final_answer
    else:
        print("Insufficient criticism from the critic agent.")
        return critic_result

In [ ]:
async with rag_mcp_server as mcp_tools: # 如果跑過workflow那段，要重新跑MCP server的建立才能跑這段
    assistant_agent.tools = [rag]
    seeker_agent.tools = [mcp_tools]
    result = await multi_turn_conversation(assistant_agent, "user_1", "session_1")

You: 我想要請假，有沒有什麼限制？
start seeker agent...
seeker agent: 您好，關於請假規定，我從公司內部規章中找到以下資訊：

**工作時間規定：**
*   每週工作時數不得超過四十小時。
*   每日工作時間為八小時，時間為 08:30 開始上班，12:00 午休，13:00 下午工作開始，15:00 下午休息，15:15 繼續工作至 17:30 下班。
*   對於未滿一歲的子女需要親自哺乳的員工，公司會提供每日額外的哺乳時間（每次 30 分鐘，視為工作時間）。
*   撫育未滿三歲子女的員工可以申請減少工作時間（每天一小時，不計報酬）或調整工作時間。

**休假種類：**
*   包含特休假、生日假、普通傷病假、事假、家庭照顧假、婚假、喪假、產假、陪產假、產檢假、安胎假、公傷假、生理假、公假共十四種。

**請假流程：**
*   員工需要事先填寫請假單或口頭說明理由，經核定後才能請假。
*   不同天數的請假需要提前不同的天數申請，並由不同層級的主管核准（0-5 天由部門主管核准，5-10 天由處級主管核准，超過 10 天由業務/事業群總經理核准）。
*   臨時狀況下，員工可以於當日 10:00 前以電話向主管報備，並事後補辦手續。
*   未依規定辦理請假手續，且事後未補辦並說明原因，將視同曠職。

**其他：**
*   公傷假之醫療費用以勞工保險及公司團保負擔，其他相關規定比照災害傷病補償辦法。
*   紀念日、勞動節日及政府規定應放假之日為放假日。

希望這些資訊對您有幫助！如果您還有其他問題，請隨時提出。
critic agent: passages=[ReferenceWithCriticism(relevant=True, text='您好，關於請假規定，我從公司內部規章中找到以下資訊：\n\n**工作時間規定：**\n*   每週工作時數不得超過四十小時。\n*   每日工作時間為八小時，時間為 08:30 開始上班，12:00 午休，13:00 下午工作開始，15:00 下午休息，15:15 繼續工作至 17:30 下班。\n*   對於未滿一歲的子女需要親自哺乳的員工，公司會提供每日額外的哺乳時間（每次 30 分鐘，視為工作時間）。\n*   撫育未滿三歲子女的員工可以申請減少工作時間（每天一小